<a href="https://colab.research.google.com/github/fernandosagastume/SPLABS/blob/master/Laboratorio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, GlobalAveragePooling1D

import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import os, re, json, functools

seed=1234
np.random.seed(seed)
tf.random.set_seed(seed)

TensorFlow 2.x selected.


In [0]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [3]:
print('train data shape:', train_data.shape)
print(train_data.shape[0], 'train samples')
print(test_data.shape[0], 'test samples')

train data shape: (25000,)
25000 train samples
25000 test samples


In [0]:
index = imdb.get_word_index()
reverse_index = dict([(value, key) for (key, value) in index.items()]) 

def decodeText(text):
  decoded = " ".join([reverse_index.get(i-3, "") for i in text])
  return decoded

In [5]:
train_data = pad_sequences(train_data, maxlen=8000)
test_data = pad_sequences(test_data, maxlen=8000)
print('Train data shape:', train_data.shape)
print('Test data shape:', test_data.shape)

Train data shape: (25000, 8000)
Test data shape: (25000, 8000)


In [0]:
model = "https://tfhub.dev/google/nnlm-en-dim128/2"
hubM = hub.KerasLayer(model, output_shape=[128], input_shape=[], 
                           dtype=tf.string, trainable=True)

In [7]:
modelK = tf.keras.Sequential()

modelK.add(hubM)
modelK.add(tf.keras.layers.Dense(1024, activation='relu'))
modelK.add(tf.keras.layers.Dense(512, activation='relu'))
modelK.add(tf.keras.layers.Dense(256, activation='relu'))
modelK.add(tf.keras.layers.Dense(128, activation='relu'))
modelK.add(tf.keras.layers.Dense(1, activation='sigmoid'))

modelK.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 128)               124642688 
_________________________________________________________________
dense (Dense)                (None, 1024)              132096    
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 125,463,937
Trainable params: 125,463,937
Non-trainable params: 0
__________________________________________

In [0]:
modelK.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [9]:
epochs = 5
train_steps = 5
validation_steps = 5 
history = modelK.fit(train_data,
                    epochs=epochs,
                    steps_per_epoch=train_steps
                   )      

ValueError: ignored